전처리 과정을 통해서 데이터셋(학습, 테스트)을 품목별로 분해하고, 품목별 데이터셋 내에서 사용자별 RFM 분석을 시도한다.

각 품목은 총 20가지로, 20가지 품목에 대해 RFM 분석을 수행하는 이유는, 특정 도메인에 대한 소비자의 패턴을 분석하기 위함이다.

각 품목별 RFM 분석에 대해서는 밀도 기반의 클러스터링 알고리즘(DBSCAN)을 적용하며, 차원축소는 품목마다 이미 데이터가 격리되었고, 스케일의 편차가 존재하지만 모수가 크지 않다고 판단되어 별도의 차원 축소는 진행하지 않는다. (이미 피처 엔지니어링이 완료됨)


In [38]:
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


# 폰트 관련 경고 메시지 숨기기
warnings.filterwarnings("ignore", category=UserWarning)
mpl.rcParams['font.family'] = 'sans-serif'

In [39]:
import os

os.path.abspath(os.curdir)

'/Users/mac/Documents/GitHub/python-training'

In [40]:
cur_dir = os.path.abspath(os.curdir)
data_dir = f"{cur_dir}/Modulabs_MainProject5"
files = [
    "orders.csv",
    "order_products__train.csv",
    "order_products__prior.csv",
    "df_products.csv"
]

df_orders, df_train, df_prior, df_products = [pd.read_csv(f"{data_dir}/{file}") for file in files]

In [41]:
df_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [42]:
df_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [43]:
df_prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [44]:
df_products

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry
...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,spirits,alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,frozen vegan vegetarian,frozen
49685,49686,Artisan Baguette,112,3,bread,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,cat food care,pets


# 문제 정의
- 품목을 고려한 고객의 RFM 분석을 실시한다.

# 워크플로우
- 품목 별로 고객 ID를 분리한다. (department_id, aisle_id로 분리할지 고민 필요)
- 품목 별로 R, F, M pandas.Series 객체를 초기화한다.
- 품목 별로 R, F, M pandas.Series 객체들을 merge 한다.
- M은 무시

In [45]:
df_products

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry
...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,spirits,alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,frozen vegan vegetarian,frozen
49685,49686,Artisan Baguette,112,3,bread,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,cat food care,pets


In [46]:
df_products_filtered = df_products[["product_id", "aisle_id", "department_id"]]
df_products_filtered

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13
...,...,...,...
49683,49684,124,5
49684,49685,42,1
49685,49686,112,3
49686,49687,41,8


In [47]:
df_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [48]:
df_train_processed = df_train.merge(df_products_filtered, on="product_id")
df_train_processed

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id
0,1,49302,1,1,120,16
1,1,11109,2,1,108,16
2,1,10246,3,0,83,4
3,1,49683,4,0,83,4
4,1,43633,5,1,95,15
...,...,...,...,...,...,...
1384612,3421063,14233,3,1,115,7
1384613,3421063,35548,4,1,13,20
1384614,3421070,35951,1,1,91,16
1384615,3421070,16953,2,1,88,13


In [49]:
df_train_processed = df_train_processed.drop(columns=["product_id", "add_to_cart_order"])
df_train_processed

,order_id,reordered,aisle_id,department_id
0,1,1,120,16
1,1,1,108,16
2,1,0,83,4
3,1,0,83,4
4,1,1,95,15
...,...,...,...,...
1384612,3421063,1,115,7
1384613,3421063,1,13,20
1384614,3421070,1,91,16
1384615,3421070,1,88,13


In [50]:
df_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


In [51]:
df_orders_analysis = df_orders[["order_id", "user_id", "days_since_prior_order"]]
df_orders_analysis

,order_id,user_id,days_since_prior_order
0,2539329,1,NaN
1,2398795,1,15.0
2,473747,1,21.0
3,2254736,1,29.0
4,431534,1,28.0
...,...,...,...
3421078,2266710,206209,29.0
3421079,1854736,206209,30.0
3421080,626363,206209,18.0
3421081,2977660,206209,7.0


In [52]:
df_train_processed = df_train_processed.merge(df_orders_analysis, on="order_id")
df_train_processed = df_train_processed.drop(columns=["order_id", "aisle_id"]) # aisle_id로도 분석은 가능하지만 너무 세부적이기 때문에 제외
df_train_processed

,reordered,department_id,user_id,days_since_prior_order
0,1,16,112108,9.0
1,1,16,112108,9.0
2,0,4,112108,9.0
3,0,4,112108,9.0
4,1,15,112108,9.0
...,...,...,...,...
1384612,1,7,169679,4.0
1384613,1,20,169679,4.0
1384614,1,16,139822,8.0
1384615,1,13,139822,8.0


In [53]:
unique_department_id = df_train_processed["department_id"].unique()
unique_department_id.sort()
unique_department_id

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21])

In [54]:
list_df = [df_train_processed[df_train_processed["department_id"] == dept_id] for dept_id in df_train_processed["department_id"].unique()]
for df in list_df:
    display(df)

,reordered,department_id,user_id,days_since_prior_order
0,1,16,112108,9.0
1,1,16,112108,9.0
7,1,16,112108,9.0
8,0,16,79431,30.0
10,0,16,79431,30.0
...,...,...,...,...
1384597,1,16,83898,11.0
1384602,1,16,136952,15.0
1384607,1,16,136952,15.0
1384610,1,16,169679,4.0


,reordered,department_id,user_id,days_since_prior_order
2,0,4,112108,9.0
3,0,4,112108,9.0
5,0,4,112108,9.0
6,0,4,112108,9.0
11,1,4,79431,30.0
...,...,...,...,...
1384583,0,4,123299,30.0
1384586,0,4,174668,30.0
1384595,0,4,189544,3.0
1384596,0,4,189544,3.0


,reordered,department_id,user_id,days_since_prior_order
4,1,15,112108,9.0
38,1,15,56463,14.0
47,1,15,56463,14.0
54,1,15,56463,14.0
55,1,15,56463,14.0
...,...,...,...,...
1384465,0,15,143793,9.0
1384495,1,15,137664,27.0
1384518,1,15,20949,21.0
1384523,1,15,20949,21.0


,reordered,department_id,user_id,days_since_prior_order
9,1,7,79431,30.0
32,1,7,56463,14.0
33,1,7,56463,14.0
52,1,7,56463,14.0
73,1,7,56463,14.0
...,...,...,...,...
1384589,0,7,174668,30.0
1384593,1,7,189544,3.0
1384599,1,7,83898,11.0
1384605,1,7,136952,15.0


,reordered,department_id,user_id,days_since_prior_order
13,1,20,79431,30.0
25,1,20,17227,30.0
46,1,20,56463,14.0
56,1,20,56463,14.0
59,1,20,56463,14.0
...,...,...,...,...
1384439,0,20,113970,30.0
1384493,1,20,137664,27.0
1384558,1,20,123299,30.0
1384566,0,20,123299,30.0


...

,reordered,department_id,user_id,days_since_prior_order
145,1,14,111860,13.0
217,0,14,61911,30.0
333,1,14,127134,7.0
379,1,14,96497,4.0
438,0,14,135489,9.0
...,...,...,...,...
1384297,1,14,142416,8.0
1384299,0,14,142416,8.0
1384472,0,14,143793,9.0
1384516,1,14,20949,21.0


,reordered,department_id,user_id,days_since_prior_order
166,0,6,184099,30.0
171,0,6,184099,30.0
186,1,6,27650,12.0
244,1,6,47475,7.0
269,1,6,164808,30.0
...,...,...,...,...
1384364,0,6,6586,5.0
1384405,1,6,69135,11.0
1384412,1,6,124642,10.0
1384477,1,6,52626,11.0


,reordered,department_id,user_id,days_since_prior_order
369,1,5,171686,4.0
915,1,5,20433,30.0
1003,1,5,19636,27.0
1298,0,5,97381,7.0
1299,0,5,97381,7.0
...,...,...,...,...
1384082,1,5,30187,30.0
1384083,0,5,30187,30.0
1384398,0,5,31068,27.0
1384399,0,5,31068,27.0


,reordered,department_id,user_id,days_since_prior_order
527,1,10,178733,3.0
2199,0,10,40556,6.0
6039,0,10,74398,30.0
6712,1,10,31638,30.0
7678,0,10,26454,30.0
...,...,...,...,...
1379468,0,10,106062,30.0
1379877,1,10,38495,4.0
1382422,1,10,97270,18.0
1383311,0,10,13440,20.0


,reordered,department_id,user_id,days_since_prior_order
690,1,8,102422,23.0
1596,0,8,107978,10.0
1810,1,8,126866,30.0
2836,1,8,85112,24.0
2837,1,8,85112,24.0
...,...,...,...,...
1382211,0,8,62014,15.0
1382254,1,8,125070,13.0
1383146,0,8,46578,15.0
1383502,1,8,51389,9.0


In [55]:
4

4